<img src="./img/ov_logo.png" style="width: 30%" />
<img src="./img/anvil_logo.png" style="width: 80%" />

## Build a machine-to-machine negotiation app in 5 minutes

<br>

Ever wanted to make an app with cryptocurrency transfers, secure comms and zero-knowledge proofs?

ANVIL lets you do just that in 5 minutes. Building on the Fetch.AI and Sovrin blockchains, the ANVIL API gives you access to powerful financial and cryptographic tools in an easy-to-understand Python package.

In this guide, you will view and edit code live. You'll learn how to build trusted, automated settlement between bots, humans and AI from scratch in just 5 minutes.

<br>


## First things first: import the ANVIL API

<br>

The ANVIL API is lightweight and imported from source:

<br>


In [1]:
from fetch.agents import offer_service, purchase_service
from sovrin.setup import setup_demo, teardown
from sovrin.onboarding import establish_channels_demo
from sovrin.schema import create_schema, create_credential_definition
from sovrin.credentials import offer_credential, receive_credential_offer, request_credential, create_and_send_credential, store_credential
from sovrin.proofs import request_proof_of_credential, create_proof_of_credential, verify_proof
import json
print('ANVIL API imported.')
# Press SHIFT + RETURN to run a code block

ANVIL API imported.


# Create the app

<br>

We are going to run through ANVIL's [certified charging station use case](https://outlierventures.io/research/anvil-a-fetch-ai-and-sovrin-bridge/).

This involves a manufacturer (issuer) certifying a charging station (prover) so that an electric car (verifier) can check that the charging station is genuine before purchasing a charge with cryptocurrency.

We will walk through four key steps:

<br>

#### 1. Create a credential

<img src="./img/manufacturer_icon.png" style="height: 15vh; display: inline-block" />
<img src="./img/arrow.png" style="height: 15vh; display: inline-block" />
<img src="./img/cred.png" style="height: 15vh; display: inline-block" />

The manufacturer will create a credential for certified charging stations.

<br><br>

#### 2. Issue the credential to a charging station

<img src="./img/cred.png" style="height: 15vh; display: inline-block" />
<img src="./img/arrow.png" style="height: 15vh; display: inline-block" />
<img src="./img/charge_icon.png" style="height: 15vh; display: inline-block; display: inline-block" />

The manufacturer will issue the certification to a charging station.

<br><br>

#### 3. Offer charging in exchange for cryptocurrency

<img src="./img/charge_icon.png" style="height: 15vh; display: inline-block; display: inline-block" />
<img src="./img/arrow.png" style="height: 15vh; display: inline-block" />
<img src="./img/buy_icon.png" style="height: 15vh; display: inline-block; display: inline-block" />

The charging station will offer a charging service in exchange for Fetch.AI tokens.

<br><br>

#### 4. Verify the charging station

<img src="./img/car_icon.png" style="height: 15vh; display: inline-block" />
<img src="./img/arrow.png" style="height: 15vh; display: inline-block" />
<img src="./img/assert.png" style="height: 15vh; display: inline-block; display: inline-block" />

The electric car will verify the charging station's credential using the Sovrin ledger before purchasing a charge.

<br>

This demo runs on the Fetch.AI testnet.

Let's get started.

<br>


In [2]:
# Set up agents
pool_name, pool_handle, steward, issuer, prover, verifier = await setup_demo()
steward, issuer, prover, verifier = await establish_channels_demo(steward, issuer, prover, verifier)



'''
MANUFACTURER: CREATE A CREDENTIAL
Specify the credential attributes in a JSON, create the schema,
create a credential definition and offer it to the charging station.
'''
schema = {
    "name": "Charging Station Certification",
    "version": "1.0",
    "attributes": ["output", "firmware", "grounded", "license", "year", "id"]
}
unique_schema_name, schema_id, issuer = await create_schema(schema, issuer)
issuer = await create_credential_definition(issuer, schema_id, unique_schema_name, revocable = False)
issuer, cred_offer = await offer_credential(issuer, unique_schema_name)



'''
CHARGING STATION: MAKE A CREDENTIAL REQUEST
The charging station receives the credential offer and fills in its
values for the credential attributes as a credential request.
'''
prover['authcrypted_cred_offer'] = cred_offer
prover = await receive_credential_offer(prover)
cred_request = json.dumps({
    "output": {"raw": "22kW", "encoded": "22"},
    "grounded": {"raw": "yes", "encoded": "1"},
    "license": {"raw": "Grounded Station", "encoded": "145"},
    "firmware": {"raw": "2.1", "encoded": "45"},
    "year": {"raw": "2019", "encoded": "2019"},
    "id": {"raw": "did:ov:37h3r", "encoded": "3708318"}
})
prover, cred_request = await request_credential(prover, cred_request)



'''
MANUFACTURER: SEND CREDENTIAL
'''
issuer['authcrypted_cred_request'] = cred_request
issuer, cred = await create_and_send_credential(issuer)



'''
CHARGING STATION: STORE CREDENTIAL AND OFFER CHARGING SERVICE
The credential is stored in the charging station's wallet.
The service is defined by JSONs listing attributes. See ANVIL-Live repo for those used here.
The price can be set as desired.
'''
prover['authcrypted_cred'] = cred
prover = await store_credential(prover)
offer_service(100, './charging_service')



'''
CAR: REQUEST PROOF OF CREDENTIAL
The car defines its desired attributes and predicates (inequalities).
'''
proof_request = json.dumps({
    "nonce": "0123456789012345678901234",
    "name": "Grounded Station proof",
    "version": "0.1",
    "requested_attributes": {
        "attr1_referent": {
            "name": "output"
        },
        "attr2_referent": {
            "name": "grounded"
        },
        "attr3_referent": {
            "name": "license"
        },
        "attr4_referent": {
            "name": "firmware"
        },
        "attr5_referent": {
            "name": "id"
        }
    },
    "requested_predicates": {
        "predicate1_referent": {
            "name": "year",
            "p_type": ">=",
            "p_value": 2019
        }
    }
})
verifier, proof_request = await request_proof_of_credential(verifier, proof_request)



'''
CHARGING STATION: CREATE PROOF
The charging station declares what it will provide itself and what
should be verified using the ledger when constructing the proof.
'''
prover['authcrypted_proof_request'] = proof_request
self_attested_attributes = {
    "attr1_referent": "22kW",
    "attr5_referent": "did:ov:37h3r"
}
prover, proof = await create_proof_of_credential(prover, self_attested_attributes, requested_attrs = [2,3,4], requested_preds = [1], non_issuer_attributes = [])



'''
CAR: VERIFY PROOF AND PURCHASE CHARGING
The car makes desired assertions on the credential attributes when verifying the proof.
In purchasing the service, the car sets a maximum price it is willing to pay.
'''
verifier['authcrypted_proof'] = proof
assertions_to_make = {
    "revealed": {
        "attr2_referent": "yes",
        "attr3_referent": "Grounded Station",
        "attr4_referent": "2.1"
    },
    "self_attested": {
        "attr1_referent": "22kW",
        "attr5_referent": "did:ov:37h3r"
    }
}
verifier = await verify_proof(verifier, assertions_to_make)
purchase_service(101, 'license_output') # 'License' and 'output' are desired properties of the OEF service



# Tear down connections
await teardown(pool_name, pool_handle, [steward, issuer, prover, verifier])
print('Negotiation complete.')

Setting up pool...
Setting up steward...
Setting up issuer...
Setting up prover...
Setting up verifier...
Steward sending connection request to issuer...
Issuer sending connection response to steward...
Steward establishing a secure channel with issuer...
Issuer getting their DID...
Steward registering issuer as a new trust anchor...
Steward sending connection request to verifier...
Verifier sending connection response to steward...
Steward establishing a secure channel with verifier...
Verifier getting their DID...
Steward registering verifier as a new trust anchor...
Issuer sending connection request to prover...
Prover sending connection response to issuer...
Issuer establishing a secure channel with prover...
Prover getting their DID...
Issuer registering prover as a new trust anchor...
Verifier sending connection request to prover...
Prover sending connection response to verifier...
Verifier establishing a secure channel with prover...
Prover getting their DID...
Verifier register


<br>

Settlement is automatic, and the charging starts. The transaction is logged to the Fetch.AI smart ledger, and both the car and charging station have established themselves as trustworthy on the Sovrin network.

<br>


## Next steps

<br>

You now know how to build a machine-to-machine negotiation app.

You can:
- [Get building](https://github.com/OutlierVentures/ANVIL),
- [Learn about who built ANVIL](https://outlierventures.io),
- [Dive into machine learning for blockchain](https://fetch.ai), or
- [become a decentralized identity pro](https://sovrin.org/).

<br>
<img src="./img/ov_logo.png" style="width: 25%" />
<center>
<a href="https://linkedin.com/in/theoturner">Written by Theo Turner</a></div>
</center>

<br>